**Library Imports**

In [1]:
import pandas as pd
import io
import requests
import numpy as np
import os
import seaborn as sns

from scipy.stats import zscore

from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier

from matplotlib import pyplot
import matplotlib.pyplot as plot
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential, load_model

from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import LabelEncoder
from IPython.display import display

from imblearn.over_sampling import SMOTE

import tensorflow as tf

Panda Preferences

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

Read CSV

In [3]:
df = pd.read_csv("../Datasets/dataset_andpre/EdgeIIot_train.csv")
display(df)

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,bwd_header_size_tot,fwd_PSH_flag_count,bwd_PSH_flag_count,flow_ACK_flag_count,fwd_pkts_payload.min,fwd_pkts_payload.max,fwd_pkts_payload.tot,fwd_pkts_payload.avg,fwd_pkts_payload.std,bwd_pkts_payload.min,bwd_pkts_payload.max,bwd_pkts_payload.tot,bwd_pkts_payload.avg,bwd_pkts_payload.std,flow_pkts_payload.min,flow_pkts_payload.max,flow_pkts_payload.tot,flow_pkts_payload.avg,flow_pkts_payload.std,fwd_iat.min,fwd_iat.max,fwd_iat.tot,fwd_iat.avg,fwd_iat.std,bwd_iat.min,bwd_iat.max,bwd_iat.tot,bwd_iat.avg,bwd_iat.std,flow_iat.min,flow_iat.max,flow_iat.tot,flow_iat.avg,flow_iat.std,payload_bytes_per_second,fwd_subflow_pkts,bwd_subflow_pkts,fwd_subflow_bytes,bwd_subflow_bytes,fwd_bulk_bytes,bwd_bulk_bytes,fwd_bulk_packets,bwd_bulk_packets,fwd_bulk_rate,bwd_bulk_rate,active.max,active.tot,active.avg,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,bwd_last_window_size,type,proto_icmp,proto_tcp,proto_udp,conn_state_OTH,conn_state_REJ,conn_state_RSTO,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR,fwd_header_size_min_0,fwd_header_size_min_8,fwd_header_size_min_20,fwd_header_size_min_24,fwd_header_size_min_32,fwd_header_size_min_40,fwd_header_size_min_44,fwd_header_size_max_0,fwd_header_size_max_8,fwd_header_size_max_20,fwd_header_size_max_24,fwd_header_size_max_32,fwd_header_size_max_40,fwd_header_size_max_44,bwd_header_size_min_0,bwd_header_size_min_8,bwd_header_size_min_20,bwd_header_size_min_24,bwd_header_size_min_32,bwd_header_size_min_40,bwd_header_size_min_44,bwd_header_size_max_0,bwd_header_size_max_8,bwd_header_size_max_20,bwd_header_size_max_24,bwd_header_size_max_32,bwd_header_size_max_40,bwd_header_size_max_44,bwd_header_size_max_52,flow_FIN_flag_count_0,flow_FIN_flag_count_1,flow_FIN_flag_count_2,flow_FIN_flag_count_3,flow_FIN_flag_count_4,flow_FIN_flag_count_5,flow_FIN_flag_count_6,flow_FIN_flag_count_7,flow_SYN_flag_count_0,flow_SYN_flag_count_1,flow_SYN_flag_count_2,flow_SYN_flag_count_3,flow_SYN_flag_count_4,flow_SYN_flag_count_5,flow_SYN_flag_count_6,flow_SYN_flag_count_7,flow_SYN_flag_count_8,flow_SYN_flag_count_9,flow_SYN_flag_count_10,flow_RST_flag_count_0,flow_RST_flag_count_1,flow_RST_flag_count_2,flow_RST_flag_count_3,flow_RST_flag_count_4,history_originator_0,history_originator_1,history_originator_2,history_originator_3,history_originator_4,history_originator_5,history_originator_6,history_responder_0,history_responder_1,history_responder_2,history_responder_3,history_responder_4,history_responder_5
0,-0.014679,-0.179829,-0.005091,0,-0.044214,-0.223505,-0.756566,-0.651330,-0.014679,-0.044214,-0.756566,-0.184151,-0.046843,-0.778440,-0.777869,-0.778499,-1.303156,-0.563255,-0.589983,-0.184780,-0.040003,-0.499835,-0.071549,-0.224057,-0.170028,-0.202208,-0.218073,-0.026854,-0.0416,-0.042557,-0.034801,-0.029821,-0.033188,-0.224624,-0.170656,-0.208086,-0.221173,0.712756,0.273386,-0.002815,0.505192,-0.366493,-0.094334,-0.191905,-0.136594,-0.176039,-0.170188,0.700756,0.258094,-0.014705,0.494734,-0.371757,-0.016764,-0.166467,-1.229442,-0.211130,-0.033546,-0.004556,-0.003094,-0.004674,-0.003342,-0.002221,-0.001499,-0.210402,-0.211092,-0.207410,-0.057970,0.567019,0.279126,0.022051,0.428915,-0.303942,-0.801662,-0.708314,-0.725170,-0.653861,udp_flood,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
1,-0.405701,-0.179829,-0.005091,0,-0.044214,-0.223505,-0.756566,-0.651330,-0.405701,-0.044214,-0.756566,-0.184151,-0.046843,-0.778416,-0.777869,-0.778487,-1.303156,-0.563255,-0.589983,-0.184780,-0.04

In [4]:
df.groupby('type')['type'].count()

type
arp_spoofing            1024
http_botnet                7
http_flood             78528
http_flood_node_red    53770
icmp_flood             24000
mqtt_flood             76917
normal                  9058
port_scanning           4523
tcp_flood              21235
udp_flood              89090
Name: type, dtype: int64

In [5]:
x_columns = df.columns.drop('type')

le = LabelEncoder()
le.fit(df["type"].values)

x = df[x_columns].values
y = df["type"].values
y = le.transform(y)

SMOTE

In [6]:
oversample = SMOTE()
df_smote = oversample.fit_resample(x, y)

In [7]:
display(df_smote)

(array([[-0.01467949, -0.17982942, -0.00509091, ...,  0.        ,
          0.        ,  0.        ],
        [-0.40570071, -0.17982942, -0.00509091, ...,  0.        ,
          0.        ,  0.        ],
        [-0.51608485, -0.17982942, -0.00509091, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [-0.5405874 , -0.17982942, -0.00509091, ...,  0.        ,
          0.        ,  0.        ],
        [-0.50663094, -0.17982942, -0.00509091, ...,  0.        ,
          0.        ,  0.        ],
        [-0.52254347, -0.17982942, -0.00509091, ...,  0.        ,
          0.        ,  0.        ]]), array([9, 9, 5, ..., 8, 8, 8]))

In [8]:
# save the DataFrame to a CSV file named "my_data.csv"
df.to_csv('IoT-23_train_smote.csv', index=False)